In [1]:
import requests, json, os
import pandas as pd
import numpy as np
import time
import sqlite3
import warnings

from pandas.io.json import json_normalize
from sqlalchemy import create_engine, inspect
from datetime import datetime
from TDA_Functions import *
from config import client_id

In [2]:
# gain the ability to see all rows/columns if desired
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# ignore warnings
warnings.filterwarnings("ignore")  

In [3]:
# create the stocks sql database
engineSpy = create_engine("sqlite:///spy.db", echo = False)
# connect to the stocks sql database
connSpy = sqlite3.connect('spy.db')
# create the ability to perform logic on the database 
s = connSpy.cursor()
# create the stock table in the stocks database
s.execute('CREATE TABLE IF NOT EXISTS spy_table '\
          '(full_date TEXT, spyMoStuck INTEGER, spyMoGreen INTEGER, spyMoAbovePivot INTEGER, '\
          'spyQtrStuck INTEGER, spyQtrGreen INTEGER, spyQtrAbovePivot INTEGER, spyMoHighQtr INTEGER, '\
          'spyYrStuck INTEGER, spyYrGreen INTEGER, spyYrAbovePivot INTEGER, spyMoHighYr INTEGER, '\
          'PRIMARY KEY (full_date, spyMoStuck, spyMoGreen, spyMoAbovePivot, spyQtrStuck, '\
          'spyQtrGreen, spyQtrAbovePivot, spyMoHighQtr, spyYrStuck, spyYrGreen, spyYrAbovePivot, spyMoHighYr))')
# commit the stock table
connSpy.commit()
# check the stock table and make sure it was correctly added to the database
print(inspect(engineSpy).get_table_names())

['spy_table']


In [4]:
# # # print all column names without quotes
# print(*final_df.columns, sep=', ')

In [5]:
# create a count to keep track of stocks accessed
count = 0
# time how long it takes for the code to run
start_time = time.time()
print(datetime.now())
# define our endpoint
endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format('SPY')
# define our payload
payload = {'apikey':client_id,
           'periodType':'year',
           'period':'20',
           'frequencyType':'daily',
           'frequency':'1',
           'needExtendedHoursData':'false'}
# make a request
content = requests.get(url = endpoint, params = payload)
# convert it to a dictionary using JSON method
data = content.json()
day_df = pd.json_normalize(data['candles'])
day_df['datetime'] = pd.to_datetime(day_df['datetime'], unit='ms')
day_df['full_date'] = pd.to_datetime(day_df['datetime']).dt.date
# create monthly identifier column 
day_df['month'] = pd.to_datetime(day_df['datetime'], format="%m%Y").dt.to_period('m')
day_df['month'] = day_df['month'].astype(str)
# create quarterly identifier column 
day_df['quarter'] = quarterMaker(day_df['month'])
# create yearly identifier column 
day_df['year'] = yearMaker(day_df['month'])
# drop the volume quarter
day_df.drop('volume', axis=1, inplace=True)
# create a useable df
day_df = day_df.set_index('datetime')
# create a quarterly df from the daily data that produces each quarter's open, high, low and close
agg_dict = {'open': 'first',
          'high': 'max',
          'low': 'min',
          'close': 'last',
          'month': 'max',  
          'quarter': 'max'}
temp_quarter_df = day_df.resample('Q', convention='end').agg(agg_dict)
# shifting columns for ability to do calcs on rows in the past
temp_quarter_df = temp_quarter_df.rename(columns={'open':'openQuarterly'})
temp_quarter_df['high1Quarterly'] = temp_quarter_df['high'].shift(periods=1)
temp_quarter_df['low1Quarterly'] = temp_quarter_df['low'].shift(periods=1)
# create df of columns from temp_quarter_df needed for temp_df merge
abr_quarter_df = temp_quarter_df[['openQuarterly', 'high1Quarterly', 'low1Quarterly', 'quarter']]
# create a yearly df from the daily data that produces each year's open, high, low and close
agg_dict = {'open': 'first',
          'high': 'max',
          'low': 'min',
          'close': 'last',
          'year': 'max'}
temp_year_df = day_df.resample('Y').agg(agg_dict)
# shifting columns for ability to do calcs on rows in the past
temp_year_df = temp_year_df.rename(columns={'open':'openYearly'})
temp_year_df['high1Yearly'] = temp_year_df['high'].shift(periods=1)
temp_year_df['low1Yearly'] = temp_year_df['low'].shift(periods=1)
# create df of columns from temp_year_df needed for temp_df merge
abr_year_df = temp_year_df[['openYearly', 'high1Yearly', 'low1Yearly', 'year']]
# create a monthly df from the daily data that produces each month's open, high, low and close
agg_dict = {'open': 'first',
          'high': 'max',
          'low': 'min',
          'close': 'last',
          'month': 'max'}
temp_month_df = day_df.resample('M').agg(agg_dict)
# shifting columns for ability to do calcs on rows in the past
temp_month_df = temp_month_df.rename(columns={'high':'highMonthly'})
temp_month_df = temp_month_df.rename(columns={'open':'openMonthly'})
temp_month_df['high1Monthly'] = temp_month_df['highMonthly'].shift(periods=1)
temp_month_df['low1Monthly'] = temp_month_df['low'].shift(periods=1)
# create df of columns from temp_month_df needed for temp_df merge
abr_month_df = temp_month_df[['openMonthly', 'highMonthly', 'high1Monthly', 'low1Monthly', 'month']]
# create new temp_df to get dayCheck desired columns
temp_df = pd.merge(day_df, abr_month_df, on='month', how='outer')
temp_df = pd.merge(temp_df, abr_quarter_df, on='quarter', how='outer')
temp_df = pd.merge(temp_df, abr_year_df, on='year', how='outer')
# make sure full_date is in str format to extrat dates
temp_df['full_date'] = temp_df['full_date'].astype(str)
# remove all nan's, if they exist, from the dataframe so the dataframe can be operated on
temp_df = temp_df[temp_df['full_date'] != "nan"]
# create day column to use in the dayCheck helper function
temp_df['day'] = temp_df['full_date'].str[-2:]
temp_df['day'] = temp_df['day'].astype(int)
# run month, quarter and yearly check functions for SPY
temp_df['monthHelper'] = monthCheckSPY(temp_df['low'], temp_df['high'], temp_df['close'], temp_df['openMonthly'], temp_df['low1Monthly'], temp_df['high1Monthly'], temp_df['month'])
temp_df['quarterHelper'] = quarterCheckSPY(temp_df['low'], temp_df['high'], temp_df['close'], temp_df['openQuarterly'], temp_df['low1Quarterly'], temp_df['high1Quarterly'], temp_df['highMonthly'], temp_df['quarter'])
temp_df['yearHelper'] = yearCheckSPY(temp_df['low'], temp_df['high'], temp_df['close'], temp_df['openYearly'], temp_df['low1Yearly'], temp_df['high1Yearly'], temp_df['highMonthly'], temp_df['year'])
# rename columns from the tuple monthHelper column
col_list = ['spyMoStuck','spyMoGreen', 'spyMoAbovePivot']
for i, col in enumerate(col_list):
    temp_df[col] = temp_df['monthHelper'].apply(lambda helper: helper[i])
temp_df = temp_df.drop('monthHelper',axis=1)
# rename columns from the tuple quarterHelper column
col_list = ['spyQtrStuck','spyQtrGreen', 'spyQtrAbovePivot', 'spyMoHighQtr']
for i, col in enumerate(col_list):
    temp_df[col] = temp_df['quarterHelper'].apply(lambda helper: helper[i])
temp_df = temp_df.drop('quarterHelper',axis=1)
# rename columns from the tuple yearHelper column
col_list = ['spyYrStuck','spyYrGreen', 'spyYrAbovePivot', 'spyMoHighYr']
for i, col in enumerate(col_list):
    temp_df[col] = temp_df['yearHelper'].apply(lambda helper: helper[i])
temp_df = temp_df.drop('yearHelper',axis=1)
# create final df
final_df = temp_df[['full_date', 'spyMoStuck', 'spyMoGreen', 'spyMoAbovePivot', 'spyQtrStuck', 'spyQtrGreen', \
                    'spyQtrAbovePivot', 'spyMoHighQtr', 'spyYrStuck', 'spyYrGreen', 'spyYrAbovePivot', 'spyMoHighYr']]
final_df.to_sql('spy_table', connSpy, if_exists='replace', index=False)
    
print("This script took", round((time.time() - start_time)/60, 2), "minutes to run.")

2021-10-29 06:26:24.296087
This script took 0.04 minutes to run.


In [6]:
spy = pd.read_sql_query('''SELECT * FROM "spy_table"''', connSpy)

In [7]:
spy.tail(20)

,full_date,spyMoStuck,spyMoGreen,spyMoAbovePivot,spyQtrStuck,spyQtrGreen,spyQtrAbovePivot,spyMoHighQtr,spyYrStuck,spyYrGreen,spyYrAbovePivot,spyMoHighYr
5016,2021-10-01,0,1,0,1,1,0,1,0,1,1,1
5017,2021-10-04,0,0,0,1,0,0,1,0,1,1,1
5018,2021-10-05,0,1,0,1,1,0,1,0,1,1,1
5019,2021-10-06,0,1,0,1,1,0,1,0,1,1,1
5020,2021-10-07,0,1,0,1,1,0,1,0,1,1,1
5021,2021-10-08,0,1,0,1,1,0,1,0,1,1,1
5022,2021-10-11,0,1,0,1,1,0,1,0,1,1,1
5023,2021-10-12,0,1,0,1,1,0,1,0,1,1,1
5024,2021-10-13,0,1,0,1,1,0,1,0,1,1,1
5025,2021-10-14,0,1,0,1,1,0,1,0,1,1,1
